<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/ml_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import glob
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

#from tqdm import tqdm
#from tqdm.notebook import tqdm
#import seaborn as sns

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor
#rom torchvision.utils import make_grid
#import torch.nn.functional as F

In [20]:
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'

txtfiles = []

import os
# traverse whole directory
for root, dirs, files in os.walk(r'/content/drive/MyDrive/ML/data/dataset/trainset/'):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))



'/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop/AnnualCrop_2004.tif'

In [21]:
def normalizeBand(band):
  band_norm = np.interp(band, (band.min(), band.max()), (0, 1))
  return band_norm

In [22]:
XImages_MS = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

i = 0
for pic in txtfiles:
  print(i)
  with rasterio.open(pic, "r") as img:
    tmp = np.moveaxis(img.read(), 0, 2)
    XImages_MS[i] = normalizeBand(tmp)
    i += 1

#Problem you need the class and the id
#Solution: Make 10 classes and then work with ids
#Problem 2: We don't have the label

Streaming output truncated to the last 5000 lines.
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835

KeyboardInterrupt: ignored

In [ ]:
ROOT_PATH = '/content/drive/MyDrive/ML/'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
DATA_PATH = os.path.join(ROOT_PATH, 'dataset')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path = '/content/drive/MyDrive/ML/data/dataset/train.csv'

data_df = pd.read_csv(file_path)

data_df.columns = data_df.columns.str.replace('path', 'image_id')
data_df.columns = data_df.columns.str.replace('Label', 'label')

print(data_df.columns)

#data_df = data_df.iloc[0]

data_df.head()

Index(['image_id', 'label'], dtype='object')


,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [ ]:
IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [ ]:
## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

## Show batches of images
def show_batch(dl):
    for images, labels in dl:
        print(images.shape)
        fig, ax = plt.subplots(figsize=(16, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
def normalize_for_display(band_data):
    """Normalize multi-spectral imagery across bands.
    The input is expected to be in HxWxC format, e.g. 64x64x13.
    To account for outliers (e.g. extremly high values due to
    reflective surfaces), we normalize with the 2- and 98-percentiles
    instead of minimum and maximum of each band.
    """
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0, 1))
    upper_perc = np.percentile(band_data, 98, axis=(0, 1))

    return (band_data - lower_perc) / (upper_perc - lower_perc)

In [ ]:
#Before appling transform we have a different shape
#Transform reshapes the image
#Added normalize because it was taken out below (maybe other way works as well)

from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform
        
    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_class = img_id.split('.')[0].split('_')[0]
        img_id = img_id.split('.')[0]+'.tif'
        path = os.path.join(self.train_dir, img_id)
        with rasterio.open(path, "r") as r:
          img = r.read([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
          img = img.astype("int64")
        img = reshape_as_image(img)
        img_nor = normalize_for_display(img)
        img_rgb = img_nor[:, :, [3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [ ]:
data_transform = transforms.Compose([
                                transforms.Resize(size=(224, 224)),
#                                 transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float), torchvision.transforms.CenterCrop(32)])

train_ds = EuroSAT(data_df, BASE_PATH)
print(len(train_ds))

27000


In [ ]:
np.shape(train_ds[0][0])

torch.Size([3, 32, 32])

In [11]:
XImages_MS[3100]

array([[[0.32372373, 0.34674674, 0.37197196, ..., 0.82742745,
         0.5805806 , 0.6398398 ],
        [0.32372373, 0.34674674, 0.37197196, ..., 0.82742745,
         0.5805806 , 0.6398398 ],
        [0.32272273, 0.32772774, 0.34174174, ..., 0.82522523,
         0.5767768 , 0.64044046],
        ...,
        [0.37217218, 0.3787788 , 0.4128128 , ..., 0.8044044 ,
         0.56996995, 0.6746747 ],
        [0.37337336, 0.37457457, 0.394995  , ..., 0.7991992 ,
         0.5695696 , 0.66986984],
        [0.37357357, 0.37197196, 0.390991  , ..., 0.78999   ,
         0.56616616, 0.6624625 ]],

       [[0.32372373, 0.34674674, 0.37197196, ..., 0.82742745,
         0.5805806 , 0.6398398 ],
        [0.32372373, 0.34674674, 0.37197196, ..., 0.82742745,
         0.5805806 , 0.6398398 ],
        [0.32272273, 0.32772774, 0.34174174, ..., 0.82522523,
         0.5767768 , 0.64044046],
        ...,
        [0.37217218, 0.3787788 , 0.4128128 , ..., 0.8044044 ,
         0.56996995, 0.6746747 ],
        [0.3

DONT CHANGE THE SHAPE OF THE IMAGE FROM TRAINING - ONLY FOR VISUALIZING BECAUSE LIBRARY LIKE rasterio use different dimensions


In [ ]:
batch_size = 1
train_ds[3100]

(tensor([[[0.8529, 0.7353, 0.7794,  ..., 1.0147, 0.8235, 0.5735],
          [0.7647, 0.5735, 0.3676,  ..., 0.7206, 0.4853, 0.5000],
          [0.6912, 0.4559, 0.4118,  ..., 0.6029, 0.6324, 0.7353],
          ...,
          [0.4265, 0.3382, 0.4706,  ..., 0.6618, 0.2500, 0.4265],
          [0.6765, 0.4853, 0.2206,  ..., 0.6618, 0.2206, 0.2647],
          [0.7500, 0.5294, 0.3676,  ..., 0.2941, 0.2206, 0.3824]],
 
         [[0.6885, 0.7951, 1.0656,  ..., 0.8115, 0.9016, 0.6885],
          [0.7623, 0.7623, 0.6557,  ..., 0.7459, 0.5902, 0.6721],
          [0.6639, 0.6148, 0.3033,  ..., 0.6967, 0.6721, 0.7377],
          ...,
          [0.4180, 0.3033, 0.3525,  ..., 0.5410, 0.4016, 0.2951],
          [0.4262, 0.4508, 0.0902,  ..., 0.3770, 0.3033, 0.4098],
          [0.5492, 0.4508, 0.3115,  ..., 0.2623, 0.2459, 0.4426]],
 
         [[0.3148, 0.7778, 0.7222,  ..., 0.9630, 0.8333, 0.4815],
          [0.6111, 0.6481, 0.5000,  ..., 0.6852, 0.4444, 0.4074],
          [0.5741, 0.3704, 0.2407,  ...,

In [ ]:
## Data loaders and showing batch of data

#show_batch(train_dl)

Model

In [ ]:
# implement the CIFAR10Net network architecture
class CIFAR10Net(nn.Module):
    
    # define the class constructor
    def __init__(self):
        
        # call super class constructor
        super(CIFAR10Net, self).__init__()
        
        # specify convolution layer 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        
        # define max-pooling layer 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify convolution layer 2
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        
        # define max-pooling layer 2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify fc layer 1 - in 16 * 5 * 5, out 120
        self.linear1 = nn.Linear(16 * 5 * 5, 120, bias=True) # the linearity W*x+b
        self.relu1 = nn.ReLU(inplace=True) # the non-linearity
        
        # specify fc layer 2 - in 120, out 84
        self.linear2 = nn.Linear(120, 84, bias=True) # the linearity W*x+b
        self.relu2 = nn.ReLU(inplace=True) # the non-linarity
        
        # specify fc layer 3 - in 84, out 10
        self.linear3 = nn.Linear(84, 10) # the linearity W*x+b
        
        # add a softmax to the last layer
        self.logsoftmax = nn.LogSoftmax(dim=1) # the softmax
        
    # define network forward pass
    def forward(self, images):
        
        # high-level feature learning via convolutional layers
        
        # define conv layer 1 forward pass
        x = self.pool1(self.relu1(self.conv1(images)))
        
        # define conv layer 2 forward pass
        x = self.pool2(self.relu2(self.conv2(x)))
        
        # feature flattening
        
        # reshape image pixels
        x = x.view(-1, 16 * 5 * 5)
        
        # combination of feature learning via non-linear layers
        
        # define fc layer 1 forward pass
        x = self.relu1(self.linear1(x))
        
        # define fc layer 2 forward pass
        x = self.relu2(self.linear2(x))
        
        # define layer 3 forward pass
        x = self.logsoftmax(self.linear3(x))
        
        # return forward pass result
        return x

In [ ]:
model = CIFAR10Net()

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [ ]:
"""def get_device():
    print(torch.cuda.is_available())
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)"""

In [ ]:
model = model.to(device)

In [ ]:
# define the optimization criterion / loss function
nll_loss = nn.NLLLoss()

In [ ]:
nll_loss = nll_loss.to(device)

In [ ]:
# define learning rate and optimization strategy
learning_rate = 0.001
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

In [ ]:
# specify the training parameters
num_epochs = 20 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

In [ ]:
train_dl = DataLoader(train_ds, batch_size=mini_batch_size, shuffle=True, num_workers=2, pin_memory=True)
#train_dl = DeviceDataLoader(train_dl, device)

In [ ]:
print(train_ds[0][0].size())

torch.Size([3, 32, 32])


In [ ]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
cifar10_train_data = torchvision.datasets.CIFAR10(root=BASE_PATH, train=True, transform=transf, download=True)
cifar10_train_dataloader = torch.utils.data.DataLoader(cifar10_train_data, batch_size=mini_batch_size, shuffle=True)

Files already downloaded and verified


In [ ]:
print(cifar10_train_data[0][0].size())

torch.Size([3, 32, 32])


In [ ]:
print(cifar10_train_data[0][0])

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [ ]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the CIFAR10 model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []

    # iterate over all-mini batches
    for i, (images, labels) in enumerate(train_dl):

        print(images)

        # push mini-batch data to computation device
        images = images.to(device)
        labels = labels.to(device)

        # run forward pass through the network
        output = model(images)
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = nll_loss(output, labels)
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())

    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # set filename of actual model
    model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))

    # save current model to GDrive models directory
    torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

tensor([[[[ 0.0965,  0.3810,  0.1015,  ...,  0.1165,  0.3258,  0.1754],
          [ 0.2231,  0.4223,  0.1541,  ...,  0.1466,  0.1404,  0.0401],
          [ 0.2318,  0.2118,  0.0789,  ...,  0.0777,  0.1165,  0.1003],
          ...,
          [ 0.4073,  0.5564,  0.7581,  ...,  0.4411,  0.3296,  0.2356],
          [ 0.4950,  0.5113,  0.6604,  ...,  0.6817,  0.2456,  0.1429],
          [ 0.4211,  0.4937,  0.6416,  ...,  0.2782,  0.0376,  0.0602]],

         [[ 0.0714,  0.1841,  0.1409,  ...,  0.0113,  0.2367,  0.1841],
          [ 0.2348,  0.4340,  0.1296,  ...,  0.1766,  0.1728,  0.1296],
          [ 0.1597,  0.1165, -0.0019,  ...,  0.0000,  0.1296,  0.1954],
          ...,
          [ 0.4603,  0.6463,  0.6819,  ...,  0.4246,  0.2142,  0.1803],
          [ 0.4283,  0.4678,  0.9280,  ...,  0.5824,  0.1634,  0.0526],
          [ 0.3795,  0.3344,  0.4603,  ...,  0.3137,  0.0376,  0.0545]],

         [[ 0.1173,  0.2851,  0.0975,  ...,  0.2166,  0.2617,  0.1570],
          [ 0.1624,  0.5414,  

KeyboardInterrupt: ignored

Visualize training loss per epoch

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# add grid
ax.grid(linestyle='dotted')

# plot the training epochs vs. the epochs' classification error
ax.plot(np.array(range(1, len(train_epoch_losses)+1)), train_epoch_losses, label='epoch loss (blue)')

# add axis legends
ax.set_xlabel("[training epoch $e_i$]", fontsize=10)
ax.set_ylabel("[Classification Error $\mathcal{L}^{NLL}$]", fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# add plot title
plt.title('Training Epochs $e_i$ vs. Classification Error $L^{NLL}$', fontsize=10);

Evaluation

In [ ]:
# restore pre-trained model snapshot
best_model_name = os.path.join(models_directory, 'cifar10_model_epoch_19.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = CIFAR10Net()

# load pre-trained models
best_model.load_state_dict(state_dict_best)

In [ ]:
# set model in evaluation mode
best_model.eval()

In [ ]:
cifar10_eval_dataloader = torch.utils.data.DataLoader(cifar10_eval_data, batch_size=10000, shuffle=False)

In [ ]:
# init collection of mini-batch losses
eval_mini_batch_losses = []

# iterate over all-mini batches
for i, (images, labels) in enumerate(cifar10_eval_dataloader):

    # run forward pass through the network
    output = best_model(images)

    # determine classification loss
    loss = nll_loss(output, labels)

    # collect mini-batch reconstruction loss
    eval_mini_batch_losses.append(loss.data.item())

# determine mean min-batch loss of epoch
eval_loss = np.mean(eval_mini_batch_losses)

# print epoch loss
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] eval-loss: {}'.format(str(now), str(eval_loss)))

Sample prediction

In [ ]:
# set (random) image id
image_id = 777

# retrieve image exhibiting the image id
cifar10_eval_image, cifar10_eval_label = cifar10_eval_data[image_id]

True class of image

In [ ]:
cifar10_classes[cifar10_eval_label]

Inspect the image

In [ ]:
# define tensor to image transformation
trans = torchvision.transforms.ToPILImage()

# set image plot title 
plt.title('Example: {}, Label: {}'.format(str(image_id), str(cifar10_classes[cifar10_eval_label])))

# un-normalize cifar 10 image sample
cifar10_eval_image_plot = cifar10_eval_image / 2.0 + 0.5

# plot cifar 10 image sample
plt.imshow(trans(cifar10_eval_image_plot))

Compare true label with prediction

In [ ]:
cifar10_eval_image.unsqueeze(0).shape
best_model(cifar10_eval_image.unsqueeze(0))

Obtain all the predictions

In [ ]:
predictions = torch.argmax(best_model(iter(cifar10_eval_dataloader).next()[0]), dim=1)

Overall classification accuracy

In [ ]:
metrics.accuracy_score(cifar10_eval_data.targets, predictions.detach())

Heatmap

In [ ]:
# determine classification matrix of the predicted and target classes
mat = confusion_matrix(cifar10_eval_data.targets, predictions.detach())

# initialize the plot and define size
plt.figure(figsize=(8, 8))

# plot corresponding confusion matrix
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='YlOrRd_r', xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.tick_params(axis='both', which='major', labelsize=8, labelbottom = False, bottom=False, top = False, left = False, labeltop=True)

# set plot title
plt.title('CIFAR-10 classification matrix')

# set plot axis lables
plt.xlabel('[true label]')
plt.ylabel('[predicted label]');